In [0]:
%sql
-- Gives unique row number to based on order_id

select row_number() over(order by order_id asc) s_no, order_id, order_date, total_amount from orders

In [0]:
%sql
-- Gives unique row number to based on order date

select 
row_number() over(partition by order_date order by order_date desc) row_num, 
order_id, 
order_date, 
total_amount 
from orders

In [0]:
%sql
-- real life use case: show first product bought by each customer

with cte_product_rank as (
    select 
        c.customer_id,
        c.first_name, 
        c.last_name,
        od.order_id, 
        od.product_name, 
        od.order_date, 
        row_number() over(partition by c.customer_id order by od.order_date desc) as product_rank
    from customers c 
        inner join order_details od
            on c.customer_id = od.customer_id)

select * from cte_product_rank where product_rank = 1

In [0]:
%sql
-- real life use case: show first product bought by each customer

with cte_product_rank as (
    select 
        c.customer_id,
        c.first_name, 
        c.last_name,
        od.order_id, 
        od.product_name, 
        od.order_date, 
        row_number() over(partition by c.customer_id order by od.order_date desc) as product_rank
    from customers c 
        inner join order_details od
            on c.customer_id = od.customer_id
        inner join orders o
            on od.order_id = o.order_id
  )

select * from cte_product_rank where product_rank = 1

In [0]:
%sql
-- based on quantity gives rank

select 
product_name,
quantity,
rank() over(order by quantity desc) as ranking
from order_details

In [0]:
%sql
-- real life use case: assign bonus based on best selling producs in each order

select 
order_id,
product_name,
quantity,
rank() over(partition by order_id order by quantity desc) as ranking
from order_details

In [0]:
%sql
-- comparing dense rank and rank

select 
product_name,
quantity,
dense_rank() over(order by quantity desc) as dense_rank,
rank() over(order by quantity desc) as rank
from order_details

In [0]:
%sql
-- real life use case: label product popularity tier in clean rank buckets

select
product_name,
sum(quantity) total_quantity,
dense_rank() over(order by sum(quantity) desc) as dense_rank
from order_details
group by product_name

In [0]:
%sql
-- lag(): compare with previous date

select
order_id,
order_date,
lag(order_date) over(order by order_date) as previous_date
from orders


In [0]:
%sql
-- lead(): compare with next date

select
order_id,
order_date,
lead(order_date) over(order by order_date) as next_date
from orders


In [0]:
%sql
-- real life use case: analyse customers order trends like they bought again

with cte_previous_quantity_check as (
select
c.first_name, o.order_id, od.product_name, o.order_date, od.quantity,
lag(od.quantity) over(partition by c.customer_id order by o.order_date) as previous_quantity
from customers c
inner join orders o
  on c.customer_id = o.customer_id
inner join order_details od
  on o.order_id = od.order_id)

select * from cte_previous_quantity_check where previous_quantity > 0


In [0]:
%sql
-- real life use case: analyse customers order trends like they didn't bought

with cte_previous_quantity_check as (
select
c.first_name, o.order_id, od.product_name, o.order_date, od.quantity,
lag(od.quantity) over(partition by c.customer_id order by o.order_date) as previous_quantity
from customers c
inner join orders o
  on c.customer_id = o.customer_id
inner join order_details od
  on o.order_id = od.order_id)

select * from cte_previous_quantity_check where previous_quantity is null


In [0]:
%sql
-- real life use case: predict what product customer might buy next

with cte_buy_next as(
  select
  c.first_name,
  o.order_id,
  od.product_name,
  lead(od.product_name) over(partition by c.customer_id order by o.order_date) as next_product
  from customers c
  inner join orders o
    on c.customer_id = o.customer_id
  inner join order_details od
    on o.order_id = od.order_id
)

select * from cte_buy_next

In [0]:
%sql
-- real life use case: predict what product customer might didn't buy next product

with cte_buy_next as(
  select
  c.first_name,
  o.order_id,
  od.product_name,
  lead(od.product_name) over(partition by c.customer_id order by o.order_date) as next_product
  from customers c
  inner join orders o
    on c.customer_id = o.customer_id
  inner join order_details od
    on o.order_id = od.order_id
)

select * from cte_buy_next where next_product is null

In [0]:
%sql
-- SUM() : Running Total OR Cummulative Sum

select 
order_id, product_id, product_name, quantity,
sum(quantity) over(order by order_id) as commulative_sum
from order_details

In [0]:
%sql
-- SUM() : Running Total OR Cummulative Sum of salary based on emp_id

with cte_salary as (
  select
  emp_id, first_name, department, job_title, salary,
  sum(salary) over(order by emp_id) salary_running_total
  from employees
)

select * from cte_salary

In [0]:
%sql
-- AVG() : Running AVG of salary based on emp_id

with cte_salary_avg as (
  select
  emp_id, first_name, department, job_title, salary,
  round(avg(salary) over(order by emp_id), 2) as salary_running_avg
  from employees
)

select * from cte_salary_avg